In [11]:
import pandas as pd
from tqdm import tqdm

In [5]:
posts = pd.read_json('../data/interim/cleared/looksource-posts-cleared.json')
posts2 = pd.read_json('../data/interim/cleared/lookconstructor-posts-cleared.json')
posts3 = pd.read_json('../data/interim/cleared/looksales-posts-cleared.json')

likes_looksource = pd.read_json('../data/raw/looksource-likes-11-Apr-2023 (14_45_50).json')
likes_looksource = likes_looksource.T.rename(columns={0:'likers'}).reset_index().rename(columns={'index':'id'})
likes_lookconstructor = pd.read_json('../data/raw/lookconstructor-likes-11-Apr-2023 (20_14_33).json')
likes_lookconstructor = likes_lookconstructor.T.rename(columns={0:'likers'}).reset_index().rename(columns={'index':'id'})
likes_looksales = pd.read_json('../data/raw/look_sales-likes-11-Apr-2023 (18_10_12).json')
likes_looksales = likes_looksales.T.rename(columns={0:'likers'}).reset_index().rename(columns={'index':'id'})

In [6]:
looksource = posts.merge(likes_looksource, on='id')
lookconstructor = posts2.merge(likes_lookconstructor, on='id')
looksales = posts3.merge(likes_looksales, on='id')

In [15]:
items = pd.concat([looksales, lookconstructor, looksource])
items.head()

,id,owner_id,date,text,likes,reposts,attachments,marked_as_ads,likers
0,24666,-196395451,2023-04-07 12:05:01,топ wb😻,97,48,[https://sun6-21.userapi.com/impg/ttKRrIk8GGMI...,0,"[233973847, 337241823, 390896289, 272906420, 4..."
1,24658,-196395451,2023-04-06 18:05:00,серьги wb🦋,76,18,[https://sun6-23.userapi.com/impg/RRRCvHmln_az...,0,"[625436056, 284492254, 566482117, 331203972, 4..."
2,24642,-196395451,2023-04-05 18:05:00,топ wb💗,216,107,[https://sun6-22.userapi.com/impg/O3fHlE6heq0x...,0,"[589773830, 460081509, 272906420, 497319001, 4..."
3,24637,-196395451,2023-04-05 16:05:00,платье wb🤤,134,141,[https://sun6-21.userapi.com/impg/TEbkmBMtxDNh...,0,"[470856355, 157540716, 530970601, 600472078, 5..."
4,24616,-196395451,2023-04-04 18:45:00,гель для бровей wb😍,205,78,[https://sun6-21.userapi.com/impg/2KajpEpvHU5R...,0,"[94330640, 600470525, 704829093, 600472078, 47..."


In [17]:
# add unique id
items['unique_id'] = items['id'].map(str) + items['owner_id'].map(str)
# drop unnecessary col
items = items.drop(columns='marked_as_ads')
items = items.reset_index(drop=True)

# drop items with unparsed likers
items = items.drop(items[items.likers.apply(len) == 0].index)

# equals likes in cols
items.likes = items.likers.apply(len).values
items = items.reset_index(drop=True)

items.head()

,id,owner_id,date,text,likes,reposts,attachments,likers,unique_id
0,24666,-196395451,2023-04-07 12:05:01,топ wb😻,206,48,[https://sun6-21.userapi.com/impg/ttKRrIk8GGMI...,"[233973847, 337241823, 390896289, 272906420, 4...",24666-196395451
1,24658,-196395451,2023-04-06 18:05:00,серьги wb🦋,101,18,[https://sun6-23.userapi.com/impg/RRRCvHmln_az...,"[625436056, 284492254, 566482117, 331203972, 4...",24658-196395451
2,24642,-196395451,2023-04-05 18:05:00,топ wb💗,263,107,[https://sun6-22.userapi.com/impg/O3fHlE6heq0x...,"[589773830, 460081509, 272906420, 497319001, 4...",24642-196395451
3,24637,-196395451,2023-04-05 16:05:00,платье wb🤤,152,141,[https://sun6-21.userapi.com/impg/TEbkmBMtxDNh...,"[470856355, 157540716, 530970601, 600472078, 5...",24637-196395451
4,24616,-196395451,2023-04-04 18:45:00,гель для бровей wb😍,212,78,[https://sun6-21.userapi.com/impg/2KajpEpvHU5R...,"[94330640, 600470525, 704829093, 600472078, 47...",24616-196395451


In [29]:
# indices with audio or video attachments
bad_idx = [615, 632, 642, 744, 805, 2954, 3655, 
           4646, 5272, 5725, 8398, 9414, 9593, 
           9829, 10107, 10117, 10218, 10624, 
           11413, 11504, 12701, 12847, 12983, 19806]

items = items.drop(bad_idx).reset_index(drop=True)

In [12]:
for i, row in tqdm(items.iterrows()):
    for j, attachment in enumerate(row['attachments']):    
        if attachment.get('photo', None) != None:
            items.iloc[i]['attachments'][j] = attachment['photo']['sizes'][-1]['url']
        else:
            del items.iloc[i]['attachments'][j]

20215it [00:30, 667.78it/s]


In [30]:
items.attachments = items.attachments.apply(str)

In [32]:
items.to_parquet('../data/processed/items.parquet.gzip', compression='gzip')